# Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import os
import nltk
import tensorflow as tf
from keras.layers import *
from tensorflow import *
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import *
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.preprocessing import LabelEncoder
import warnings
import csv
import random
from collections import Counter
from utils.textnormalization import TextNormalization
from utils.machinelearning import machine_learning_model, fit_model, metrics_values, confusion_matrix_
from utils.deeplearning import convolutional_neural_network_1d, model_compile, model_fit, evaluate, plot_accuracy_loss, lstm_, gru_
from utils.sentimentinformation import sentiment_percentage, sentiment_counts
from utils.wordcloud import wordcloud
from utils.most_frequent_word_plot import go_figure , bar_plot
from utils.deeplearning_preprcosesing import  deeplearning_preprcosesing_
pd.options.display.max_rows = 100
pd.options.display.max_columns =None 
random.seed(40)
tf.random.set_seed(40)

# Download Some Packages

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Ignore Warnings

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings(action='ignore')

# Read Data

In [ ]:
Train = pd.read_csv('train.csv')
Train.rename(columns={'text': 'Review', 'sentiment': 'Sentiment'}, inplace=True)
Train['Sentiment'] = Train['Sentiment'].map({'neg': 'Negative', 'pos': 'Positive'})
Train.head(5)

In [ ]:
Test = pd.read_csv('test.csv')
Test.rename(columns={'text': 'Review', 'sentiment': 'Sentiment'}, inplace=True)
Test['Sentiment'] = Test['Sentiment'].map({'neg': 'Negative', 'pos': 'Positive'})
Test.head(5)

# Sentiment Information

**1-Train Set**

In [ ]:
sentiment_percentage(Train, 'Sentiment', (6, 6))

In [ ]:
sentiment_counts(Train, 'Sentiment', (6, 6))

**2-Test Set**

In [ ]:
sentiment_percentage(Test, 'Sentiment', (6, 6))

In [ ]:
sentiment_counts(Test, 'Sentiment', (6, 6))

# Number Of Sentences

**1-Train**

In [ ]:
Train['NumberOfSentences'] = Train['Review'].apply(lambda n:len(sent_tokenize(n)))
Train.drop(Train[Train["NumberOfSentences"] == 0].index, inplace = True)


In [ ]:
Train['NumberOfSentences'].sum()

**2-Test**

In [ ]:
Test['NumberOfSentences'] = Test['Review'].apply(lambda n:len(sent_tokenize(n)))
Test.drop(Test[Test["NumberOfSentences"] == 0].index, inplace = True)

In [ ]:
Test['NumberOfSentences'].sum()

# Average Number Of Sentences

**1-Train Set**

In [ ]:
AverageNumberOfSentence_Train = int(np.round(Train["NumberOfSentences"].mean()))
AverageNumberOfSentence_Train

**2-Test Set**

In [ ]:
AverageNumberOfSentence_Test = int(np.round(Test["NumberOfSentences"].mean()))
AverageNumberOfSentence_Test

# Maximum Number Of Sentences

**1-Train Set**

In [ ]:
MaximumNumberOfSentence_Train = Train["NumberOfSentences"].max()
MaximumNumberOfSentence_Train

**2-Test Set**

In [ ]:
MaximumNumberOfSentence_Test = Test["NumberOfSentences"].max()
MaximumNumberOfSentence_Test

# Minimum Number Of Sentences

**1-Train Set**

In [ ]:
MinimumNumberOfSentence_Train = Train["NumberOfSentences"].min()
MinimumNumberOfSentence_Train

**2-Test Set**

In [ ]:
MinimumNumberOfSentence_Test = Test["NumberOfSentences"].min()
MinimumNumberOfSentence_Test

# Text Normalization

In [ ]:
text_normalization = TextNormalization(_string_lower = True, 
                        _remove_emojis = True, 
                        _remove_hashtags = True, 
                        _remove_emails = True,
                        _remove_url = True, 
                        _remove_mention = True, 
                        _remove_duplicate_char = True,
                        _remove_single_char = True, 
                        _remove_new_line_char = True, 
                        _remove_number = True, 
                        _remove_html_tags = True, 
                        _remove_special_character = True, 
                        _remove_longest_than = True, 
                        _remove_whitespace = True, 
                        _remove_unicode_characters = True,
                        _stemmer = False, 
                        _remove_non_english = True, 
                        _remove_stop_words = True, 
                        _lemmatizer = False)

**1-Train Set**

In [ ]:
Train['Normalized_Review'] = Train['Review'].apply(lambda x: text_normalization.normalization(x))
Train

**2-Test Set**

In [ ]:
Test['Normalized_Review'] = Test['Review'].apply(lambda x: text_normalization.normalization(x))
Test

# Number Of Unique Words

**1-Train Set**

In [ ]:
UniqueWords_Train = set()
for word in Train['Normalized_Review'].str.split():
    UniqueWords_Train.update(word)
len(UniqueWords_Train)

**2-Test Set**

In [ ]:
UniqueWords_Test = set()
for word in Test['Normalized_Review'].str.split():
    UniqueWords_Test.update(word)
len(UniqueWords_Test)

# WordCloud

**1-Train Set**

In [ ]:
wordcloud(data=Train, target='Negative', width=1500, hieght=800, text='Normalized_Review')

In [ ]:
wordcloud(data=Train, target='Positive', width=1500, hieght=800, text='Normalized_Review')

**2-Test Set**

In [ ]:
wordcloud(data=Test, target='Negative', width=1500, hieght=800, text='Normalized_Review')

In [ ]:
wordcloud(data=Test, target='Positive', width=1500, hieght=800, text='Normalized_Review')

# Most Frequent Words 

**1-Train Set**

In [ ]:
MostFrequentWord_Train = Counter((Train['Normalized_Review']).str.split().explode()).most_common(15)
MostFrequentWordDF_Train = pd.DataFrame(MostFrequentWord_Train, columns=('Word', 'Count'))
MostFrequentWordDF_Train

In [ ]:
go_figure(data=MostFrequentWordDF_Train)

In [ ]:
bar_plot(data=MostFrequentWordDF_Train)

**2-Test Set**

In [ ]:
MostFrequentWord_Test = Counter((Test['Normalized_Review']).str.split().explode()).most_common(15)
MostFrequentWordDF_Test = pd.DataFrame(MostFrequentWord_Test, columns=('Word', 'Count'))
MostFrequentWordDF_Test

In [ ]:
go_figure(data=MostFrequentWordDF_Test)

In [ ]:
bar_plot(data=MostFrequentWordDF_Test)

# Number Of Words 

**1-Train Set**

In [ ]:
Train["Number of Words"] = Train["Normalized_Review"].apply(lambda n:len(word_tokenize(n)))
Train.drop(Train[Train["Number of Words"] == 0].index, inplace = True)
Train["Number of Words"].sum()

**2-Test Set**

In [ ]:
Test["Number of Words"] = Test["Normalized_Review"].apply(lambda n:len(word_tokenize(n)))
Test.drop(Train[Test["Number of Words"] == 0].index, inplace = True)
Test["Number of Words"].sum()

# Number Of Charachters 

**1-Train Test**

In [ ]:
Train["Number of Charachters"] = Train["Normalized_Review"].str.len()
Train["Number of Charachters"].sum()

**2-Test Set**

In [ ]:
Test["Number of Charachters"] = Test["Normalized_Review"].str.len()
Test["Number of Charachters"].sum()

# Average Number Of Words 

**1-Train Set**

In [ ]:
AverageNaumberOfWords_Train = int(np.round((Train["Number of Words"].mean())))
AverageNaumberOfWords_Train

**2-Test Set**

In [ ]:
AverageNaumberOfWords_Test = int(np.round((Test["Number of Words"].mean())))
AverageNaumberOfWords_Test

# Average Number Of Charachters 

**1-Train Set**

In [ ]:
AverageNumberOfCharachters_Train = int(np.round(Train["Number of Charachters"].mean()))
AverageNumberOfCharachters_Train

**2-Test Set**

In [ ]:
AverageNumberOfCharachters_Test = int(np.round(Test["Number of Charachters"].mean()))
AverageNumberOfCharachters_Test

# Maximum Number Of Word

**1-Train Set**

In [ ]:
MaximumNaumberOfWords_Train = Train["Number of Words"].max()
MaximumNaumberOfWords_Train

**2-Test Set**

In [ ]:
MaximumNaumberOfWords_Test = Test["Number of Words"].max()
MaximumNaumberOfWords_Test

# Maximum Number Of Charachters 

**1-Train Set**

In [ ]:
MaximumNumberOfCharachters_Train = Train['Number of Charachters'].max() 
MaximumNumberOfCharachters_Train

**2-Test Set**

In [ ]:
MaximumNumberOfCharachters_Test = Test['Number of Charachters'].max() 
MaximumNumberOfCharachters_Test

# Minimum Number Of Words

**1-Train Set**

In [ ]:
MimimumNaumberOfWords_Train = Train["Number of Words"].min()
MimimumNaumberOfWords_Train

**2-Test Set**

In [ ]:
MimimumNaumberOfWords_Test = Test["Number of Words"].min()
MimimumNaumberOfWords_Test

# Minimum Number Of Charachters 

**1-Train Set**

In [ ]:
MinimumNumberOfCharachters_Train = Train['Number of Charachters'].min() 
MinimumNumberOfCharachters_Train

**2-Test Set**

In [ ]:
MinimumNumberOfCharachters_Test = Test['Number of Charachters'].min() 
MinimumNumberOfCharachters_Test

# Split The Dataset Into Training And Testing

In [ ]:
X_train, X_test, y_train, y_test = Train['Normalized_Review'], Test['Normalized_Review'], Train['Sentiment'], Test['Sentiment']

# Encoding Dependent Variables (Y) Into Integer

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# CountVectorizer

In [ ]:
CountVectorizer_ = CountVectorizer()
X_train_CV_ = CountVectorizer_.fit_transform(X_train)
X_test_CV_ = CountVectorizer_.transform(X_test)

In [ ]:
Voc = CountVectorizer_.vocabulary_
Voc

# Machine Learning

**1-Naive Bayes**

In [ ]:
Algorithm = MultinomialNB(alpha= 1)
Naive_Bayes = machine_learning_model(algorithm=Algorithm)
fit_model(model=Naive_Bayes, X_train=X_train_CV_, y_train=y_train)
confusion_matrix_(model=Naive_Bayes, X_test=X_test_CV_, y_test=y_test, name='Naive Bayes')

In [ ]:
metrics_values(model= Naive_Bayes, X_test=X_test_CV_, y_test=y_test)

**2-Logistic Regression**

In [ ]:
Algorithm = LogisticRegression()
Logistic_Regression = machine_learning_model(algorithm=Algorithm)
fit_model(model=Logistic_Regression, X_train=X_train_CV_, y_train=y_train)
confusion_matrix_(Logistic_Regression, X_test=X_test_CV_, y_test=y_test, name='Logistic Regression')

In [ ]:
metrics_values(model= Logistic_Regression, X_test=X_test_CV_, y_test=y_test)

**3-Decision Tree**

In [ ]:
Algorithm = DecisionTreeClassifier(criterion='gini')
Decision_Tree = machine_learning_model(algorithm=Algorithm)
fit_model(model=Decision_Tree, X_train=X_train_CV_, y_train=y_train)
confusion_matrix_(model=Decision_Tree, X_test=X_test_CV_, y_test=y_test, name='Decision Tree')

In [ ]:
metrics_values(model= Decision_Tree, X_test=X_test_CV_, y_test=y_test)

**4-K Nearest Neighbors**

In [ ]:
Algorithm =KNeighborsClassifier(metric='manhattan', n_neighbors=5)
K_Nearest_Neighbors = machine_learning_model(algorithm=Algorithm)
fit_model(model=K_Nearest_Neighbors, X_train=X_train_CV_, y_train=y_train)
confusion_matrix_(model=K_Nearest_Neighbors, X_test=X_test_CV_, y_test=y_test, name='K Nearest Neighbors')

In [ ]:
metrics_values(model= K_Nearest_Neighbors, X_test=X_test_CV_, y_test=y_test)

**5-Random Forest**

In [ ]:
Algorithm = RandomForestClassifier(criterion = 'entropy', n_estimators=300)
Random_Forest = machine_learning_model(algorithm=Algorithm)
fit_model(model=Random_Forest, X_train=X_train_CV_, y_train=y_train)
confusion_matrix_(model=Random_Forest, X_test=X_test_CV_, y_test=y_test, name='Random Forest')

In [ ]:
metrics_values(model= Random_Forest, X_test=X_test_CV_, y_test=y_test)

**6-SGD Classifier**

In [ ]:
Algorithm = SGDClassifier(loss='hinge')
SGD = machine_learning_model(algorithm=Algorithm)
fit_model(model=SGD, X_train=X_train_CV_, y_train=y_train)
confusion_matrix_(model=SGD, X_test=X_test_CV_, y_test=y_test, name='SGD Classifier')

In [ ]:
metrics_values(model= SGD, X_test=X_test_CV_, y_test=y_test)

**7-Multilayer Perceptron**

In [ ]:
Algorithm = MLPClassifier(hidden_layer_sizes=16)
MLP = machine_learning_model(algorithm=Algorithm)
fit_model(model=MLP, X_train=X_train_CV_, y_train=y_train)
confusion_matrix_(model=MLP, X_test=X_test_CV_, y_test=y_test, name='Multilayer Perceptron')

In [ ]:
metrics_values(model= MLP, X_test=X_test_CV_, y_test=y_test)

# Deep Learning

In [ ]:
embedding_dim = 32
vocab_size, max_length, encoded_X_train , encoded_X_test = deeplearning_preprcosesing_(X_train=X_train, X_test=X_test,  truncating='pre', padding='pre')

In [ ]:
print('The Vocab Size Is :',vocab_size)
print('')
print('The Max Length Is :', max_length)

In [ ]:
Callback = tf.keras.callbacks.EarlyStopping(
                    monitor="accuracy",
                    mode="max",
                    patience=3
                )

**8-Convolutional Neural Network**

In [ ]:
convolutional_neural_network = convolutional_neural_network_1d(vocab_size=vocab_size + 1, embedding_dim=embedding_dim, max_length=max_length, dropout=0.6, kernel=2, filters=32, strides=3, padding='same')
model_compile(model=convolutional_neural_network)
history = model_fit(model=convolutional_neural_network, X_train=encoded_X_train, y_train=y_train, epochs=10, X_test=encoded_X_test, y_test=y_test, batch_size=32, Callback=Callback, shuffle=True)
plot_accuracy_loss(histoty=history)
evaluate(model=convolutional_neural_network, x=encoded_X_train, y=y_train, train_test='Training')
evaluate(model=convolutional_neural_network, x=encoded_X_test, y=y_test, train_test='Testing')

**9-LSTM**

In [ ]:
LSTM =lstm_(vocab_size= vocab_size + 1, embedding_dim= embedding_dim, max_length= max_length, dropout=0.5, units=64)
model_compile(model=LSTM)
history = model_fit(model=LSTM, X_train=encoded_X_train, y_train=y_train, epochs=5, X_test=encoded_X_test, y_test=y_test, batch_size=64, Callback=Callback, shuffle=True)
plot_accuracy_loss(histoty=history)
evaluate(model=LSTM, x=encoded_X_train, y=y_train, train_test='Training')
evaluate(model=LSTM, x=encoded_X_test,y= y_test, train_test='Testing')

**10-GRU**

In [ ]:
GRU_=gru_(vocab_size= vocab_size + 1, embedding_dim= embedding_dim, max_length= max_length, dropout=0.3, units=64)
model_compile(model=GRU_)
history = model_fit(model=GRU_, X_train=encoded_X_train, y_train=y_train, epochs=5, X_test=encoded_X_test, y_test=y_test, batch_size=64, Callback=Callback, shuffle=True)
plot_accuracy_loss(histoty=history)
evaluate(model=GRU_, x=encoded_X_train, y=y_train, train_test='Training')
evaluate(model=GRU_, x=encoded_X_test,y= y_test, train_test='Testing')  

* In Future Work I Wiil Try To Decrease The Loss Values
* Using Transformer